In [1]:
import pandas as pd
import json
from google.colab import drive

from os import listdir
from os.path import isfile, join




# Extract the evidence sentences from the given Wikipedia pages

In [2]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

onlyfiles = [f for f in listdir('/content/drive/MyDrive/wiki-pages/') if isfile(join('/content/drive/MyDrive//wiki-pages/', f))]

In [ ]:
evidences = dict()
evidences['id'] = list()
evidences['lines'] = list()
for f in onlyfiles:
    file  = '/content/drive/MyDrive/wiki-pages/'+f
    with open(file,'r') as f_:
        
        for l in f_:
            l_ = json.loads(l)
#             print(type(l))
#             print(type(l_))
            data = json.loads(l.rstrip("\n"))
            evidences['id'].append(data['id'])
            evidences['lines'].append(data["lines"].split('\n'))
              # break
            
  

KeyboardInterrupt: ignored

In [ ]:
df_evidences = pd.DataFrame.from_records(evidences)

In [ ]:
df_evidences.head(5)

In [ ]:
df_evidences[df_evidences['id'] == 'Sin_Sukju']

In [ ]:
df_evidences.to_csv('evidences_from_wikipages.csv')

In [ ]:
!cp evidences_from_wikipages.csv "drive/My Drive/"

In [3]:
train_data = pd.read_json('/content/drive/MyDrive/train.jsonl',lines = True)

df_evidences = pd.read_csv('/content/drive/MyDrive/evidences_from_wikipages.csv')

In [4]:
train_data.head(10)

,id,verifiable,label,claim,evidence
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [..."
1,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, Roman_Atwood, 1]], [[174271..."
2,214861,VERIFIABLE,SUPPORTS,"History of art includes architecture, dance, s...","[[[255136, 254645, History_of_art, 2]]]"
3,156709,VERIFIABLE,REFUTES,Adrienne Bailon is an accountant.,"[[[180804, 193183, Adrienne_Bailon, 0]]]"
4,83235,NOT VERIFIABLE,NOT ENOUGH INFO,System of a Down briefly disbanded in limbo.,"[[[100277, None, None, None]]]"
5,129629,VERIFIABLE,SUPPORTS,Homeland is an American television spy thrille...,"[[[151831, 166598, Homeland_-LRB-TV_series-RRB..."
6,149579,NOT VERIFIABLE,NOT ENOUGH INFO,Beautiful reached number two on the Billboard ...,"[[[173384, None, None, None]]]"
7,229289,NOT VERIFIABLE,NOT ENOUGH INFO,Neal Schon was named in 1954.,"[[[273626, None, None, None]]]"
8,33078,VERIFIABLE,SUPPORTS,The Boston Celtics play their home games at TD...,"[[[49158, 58489, Boston_Celtics, 3]], [[49159,..."
9,6744,VERIFIABLE,SUPPORTS,The Ten Commandments is an epic film.,"[[[23513, 28977, The_Ten_Commandments_-LRB-195..."


In [5]:
print('Training Samples: ',len(train_data))

Training Samples:  145449


# Things to do before submission:

- Basic data exploration
- Evidence sentence extraction from the wikipedia pages
- Based on claims and evidence, tell if the claim is verifiable
- Based on claim, evidence and verifiable, check the label
- Start with basic BiLSTM and GRU architectures

In [6]:
print('Maximum length of the claim in the train data: ',max(len(d) for d in train_data.claim.values))

Maximum length of the claim in the train data:  614


In [7]:
print('Minimum length of the claim in the train data: ',min(len(d) for d in train_data.claim.values))

Minimum length of the claim in the train data:  10


In [8]:
print('Maximum length of the evidence in the train data: ',max(len(d) for d in train_data.evidence.values))

Maximum length of the evidence in the train data:  251


In [9]:
print('Minimum length of the evidence in the train data: ',min(len(d) for d in train_data.evidence.values))

Minimum length of the evidence in the train data:  1


In [10]:
train_data.label.value_counts()

SUPPORTS           80035
NOT ENOUGH INFO    35639
REFUTES            29775
Name: label, dtype: int64

In [11]:
train_data.verifiable.value_counts()

VERIFIABLE        109810
NOT VERIFIABLE     35639
Name: verifiable, dtype: int64

In [12]:
train_data.evidence.values[0]

[[[92206, 104971, 'Nikolaj_Coster-Waldau', 7],
  [92206, 104971, 'Fox_Broadcasting_Company', 0]]]

In [13]:
train_evidence_id = train_data['evidence'].apply(lambda x: [k[2] for k in x[0]])

In [14]:
train_evidence_sentence_id = train_data['evidence'].apply(lambda x: [k[3] for k in x[0]])

In [15]:
from numpy import int64
train_data['evidence_id'] = train_evidence_id

# train_data['evidence_id'] = pd.to_numeric(train_data['evidence_id'])# .astype(df_evidences['id']) #= int64

# train_data.convert_objects(convert_numeric=True)

train_data['evidence_sentence_id'] = train_evidence_sentence_id

In [16]:
train_data = train_data.explode(['evidence_id','evidence_sentence_id'])

In [17]:
merged_train = pd.merge(train_data, df_evidences, how='left',left_on = 'evidence_id',right_on = 'id')

In [18]:
merged_train.head(5)

,id_x,verifiable,label,claim,evidence,evidence_id,evidence_sentence_id,Unnamed: 0,id_y,lines
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [...",Nikolaj_Coster-Waldau,7,2042353.0,Nikolaj_Coster-Waldau,['0\tNikolaj Coster-Waldau -LRB- -LSB- neɡ̊ola...
1,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [...",Fox_Broadcasting_Company,0,995034.0,Fox_Broadcasting_Company,['0\tThe Fox Broadcasting Company -LRB- often ...
2,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, Roman_Atwood, 1]], [[174271...",Roman_Atwood,1,5138676.0,Roman_Atwood,"['0\tRoman Bernard Atwood -LRB- born May 28 , ..."
3,214861,VERIFIABLE,SUPPORTS,"History of art includes architecture, dance, s...","[[[255136, 254645, History_of_art, 2]]]",History_of_art,2,3924051.0,History_of_art,['0\tThe history of art is the history of any ...
4,156709,VERIFIABLE,REFUTES,Adrienne Bailon is an accountant.,"[[[180804, 193183, Adrienne_Bailon, 0]]]",Adrienne_Bailon,0,4586736.0,Adrienne_Bailon,['0\tAdrienne Eliza Houghton -LRB- née Bailon ...


In [19]:
merged_train.verifiable.value_counts()

VERIFIABLE        128148
NOT VERIFIABLE     95218
Name: verifiable, dtype: int64

In [20]:
merged_train.drop('id_y',axis =1,inplace = True)

In [21]:
merged_train.drop('Unnamed: 0',axis =1,inplace = True)


In [22]:

merged_train.head()

,id_x,verifiable,label,claim,evidence,evidence_id,evidence_sentence_id,lines
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [...",Nikolaj_Coster-Waldau,7,['0\tNikolaj Coster-Waldau -LRB- -LSB- neɡ̊ola...
1,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [...",Fox_Broadcasting_Company,0,['0\tThe Fox Broadcasting Company -LRB- often ...
2,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, Roman_Atwood, 1]], [[174271...",Roman_Atwood,1,"['0\tRoman Bernard Atwood -LRB- born May 28 , ..."
3,214861,VERIFIABLE,SUPPORTS,"History of art includes architecture, dance, s...","[[[255136, 254645, History_of_art, 2]]]",History_of_art,2,['0\tThe history of art is the history of any ...
4,156709,VERIFIABLE,REFUTES,Adrienne Bailon is an accountant.,"[[[180804, 193183, Adrienne_Bailon, 0]]]",Adrienne_Bailon,0,['0\tAdrienne Eliza Houghton -LRB- née Bailon ...


In [26]:
merged_train.lines.isnull().value_counts()

False    222391
True        975
Name: lines, dtype: int64

In [28]:
merged_train = merged_train.fillna({'claim':'No claim', 'evidence': 'No evidence', 'evidence_id':'nil','evidence_sentence_id':-1,'lines':''})

In [30]:
merged_train.lines.isnull().value_counts()

False    223366
Name: lines, dtype: int64

In [36]:
print(len(lines),len(sentence_ids))

223366 223366


In [38]:
import ast
lines = merged_train['lines'].values.tolist()
sentence_ids = merged_train['evidence_sentence_id'].values.tolist()
evidence_text = []

for l,s in zip(lines,sentence_ids):
  if not l:
    evidence_text.append('')
  else:
    l_ = ast.literal_eval(l)
      # print('hi')
    if s!=-1:
      evidence_text.append(l_[int(s)]) 
    else:
      evidence_text.append('')
merged_train['evidence_text'] = evidence_text

In [39]:
merged_train = merged_train.groupby('id_x').agg(list)
# merged_train.dropna(inplace=True)

In [40]:
merged_train.head(5)['verifiable']

id_x
3                        [VERIFIABLE]
4    [NOT VERIFIABLE, NOT VERIFIABLE]
7                        [VERIFIABLE]
8                        [VERIFIABLE]
9                        [VERIFIABLE]
Name: verifiable, dtype: object

In [41]:
final_train = merged_train.copy()

In [42]:
final_train['verifiable'] = final_train['verifiable'].apply(lambda x: x[0])
final_train['claim'] = final_train['claim'].apply(lambda x: x[0])
final_train['evidence'] = final_train['evidence'].apply(lambda x: x[0])
final_train['evidence_id'] = final_train['evidence_id']#.apply(lambda x: x[0])
final_train['evidence_sentence_id'] = final_train['evidence_sentence_id']#.apply(lambda x: x[0] if x[0]!=None else x[0])
final_train['lines'] = final_train['lines'].apply(lambda x: x[0])
final_train['label'] = final_train['label'].apply(lambda x: x[0])
# final_train['evidence_text'] = final_train['evidence_text'].apply(lambda x: x[0])
# final_train.drop('id_y',axis =1,inplace = True)


 

final_train.head(5)

,verifiable,label,claim,evidence,evidence_id,evidence_sentence_id,lines,evidence_text
id_x,,,,,,,,
3,VERIFIABLE,SUPPORTS,Chris Hemsworth appeared in A Perfect Getaway.,"[[[15732, 19585, Chris_Hemsworth, 2]]]",[Chris_Hemsworth],[2],['0\tChris Hemsworth -LRB- born 11 August 1983...,[2\tHemsworth has also appeared in the science...
4,NOT VERIFIABLE,NOT ENOUGH INFO,Chris Hemsworth disappeared in A Perfect Getaway.,"[[[15230, None, None, None]]]","[nil, nil]","[-1, -1]","['0\tIn computing , NaN , standing for not a n...","[, ]"
7,VERIFIABLE,SUPPORTS,Roald Dahl is a writer.,"[[[15234, 18987, Roald_Dahl, 0]]]",[Roald_Dahl],[0],['0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _...,[0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ...
8,VERIFIABLE,REFUTES,Roald Dahl is a governor.,"[[[15233, 18986, Roald_Dahl, 0]]]",[Roald_Dahl],[0],['0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _...,[0\tRoald Dahl -LRB- -LSB- langpronˈroʊ.əld _ ...
9,VERIFIABLE,SUPPORTS,Ireland has relatively low-lying mountains.,"[[[15235, 18988, Ireland, 10]]]",[Ireland],[10],['0\tIreland -LRB- -LSB- ˈaɪərlənd -RSB- Éire ...,[10\tThe island 's geography comprises relativ...


In [43]:
# final_train.dropna(inplace = True)


In [45]:
final_train.verifiable.value_counts()

VERIFIABLE        109810
NOT VERIFIABLE     35639
Name: verifiable, dtype: int64

In [46]:
# final_train.drop(['Unnamed: 0'],axis =1)

In [47]:
final_train.to_csv('train_data_final.csv')

In [ ]:
!cp train_data_final.csv "drive/My Drive/"
